In [ ]:
import importlib
import utilities

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mplcursors
import pandas as pd
from matplotlib.lines import Line2D
from scipy.stats import linregress 
from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq
import scipy.signal as sig
from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq
import sys
import pickle


In [ ]:
#define the Data class: 

In [ ]:
importlib.reload(utilities)

class Data:
    def __init__(self,lcnumber):
        self.lcnumber=lcnumber
        self.datatable=None
        self.dataclass=None
        self.__load()
        self.cumsumLow=np.cumsum(self.getcolumn("low"))
        self.cumsumMid=np.cumsum(self.getcolumn("mid"))
        self.cumsumHigh=np.cumsum(self.getcolumn("high"))
        self._alpha={"low":None,"mid":None,"high":None}
        self._beta={"low":None,"mid":None,"high":None}

    def _findalpha(self,column):
         self._alpha[column]=utilities.findalpha(self.getcolumn(column))

    def _findbeta(self,column):
        self._beta[column]=utilities.findbeta(self.getcolumn(column))
    
    def alpha(self,column):
        if(self._alpha[column] is None):
            self._findalpha(column)
        return self._alpha[column]

    def beta(self,column):
        if(self._beta[column] is None):
            self._findbeta(column)
        return self._beta[column]
    
    def __load(self):
        self.datatable=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="	",skiprows=[0, 1], header=None)
        self.datatable.columns=['time', 'total','low','mid','high'] 
        
        firstline=pd.read_csv("classified_lcs/grs1915_lc"+str(self.lcnumber)+".txt",sep="	",nrows=1, header=None).iloc[0,0]
        prefix="# class : "
        self.dataclass=firstline[len(prefix):].strip()

    
    def low(self):
        return np.array(self.datatable["low"])

    def mid(self):
        return np.array(self.datatable["mid"])

    def high(self):
        return np.array(self.datatable["high"])

    def getcolumn(self,column):
        if(column=="low"):
            return self.low()
        if(column=="mid"):
            return self.mid()
        if(column=="high"):
            return self.high()
            
    def cumsum(self,column):
        if(column=="low"):
            return self.cumsumLow
        if(column=="mid"):
            return self.cumsumMid
        if(column=="high"):
            return self.cumsumHigh
        

In [ ]:
#define Shaplet class

In [ ]:
importlib.reload(utilities)
class Shaplet:
    def __init__(self,lcnumber,column,start,length,series=None):
        if not (series is None):
            self.series=series
            self.lcnumber=-1
            self.column=-1
            self.start=-1
            self.length=len(self.series)
        else:
            self.lcnumber=lcnumber
            self.start=start
            self.column=column
            self.length=length
            self.end=start+length
            self.series=None
            self.load(Data(self.lcnumber))
        self.sum=np.sum(self.series)
        self.alpha=utilities.findalpha(self.series)
        self.beta=utilities.findbeta(self.series)
        
    def normalizedseries():
        return self._alpha*self.seris+self._beta
                  
    def __len__(self):
        return self.length

    #i will leave it right now public,we will see what we can do in the future
    def load(self,data:Data):
        if data.lcnumber != self.lcnumber:
            raise ValueError("Error: lcnumber values do not match.")  # Raising an error properly
        if(self.series is None):
            self.series=data.getcolumn(self.column)[self.start:self.end]

In [ ]:
importlib.reload(utilities)
class Comparator:
    def __init__(self,data:Data,column,shaplet:Shaplet):
        self.data=data
        self.shaplet=shaplet
        self.column=column
        self.signallength=len(data.getcolumn(column))
        
        
        self._chi2=None
        self._minimaspositions=None
        self._minimas=None
        
        self._maxCut=None
        self._minCut=None
        self._efficientCut=None
        self.cumsum=self.data.cumsum(self.column)
        self.alpha=np.ones(max(len(self.cumsum) - len(self.shaplet)+1,0))
        self._scaledchi2=None
    
    def __computealpha(self):
        s=self.cumsum
        signal=self.data.getcolumn(self.column)
        N=len(self.shaplet)
        alpha=np.array([s[N-1]])
        alpha=np.concatenate((alpha,self.cumsum[N:]-self.cumsum[:-N]))
        alpha=alpha/np.sum(self.shaplet.series) 
        return alpha

    def scaledchi2measure(self,signal,shapletseries):
        #NEED TO CHECK IF THE SHAPLET SERIES IS LOADED IF NOT THEN I LL HAE AN ERROR 
        if(len(signal)<len(shapletseries)): #By convention the difference is -1 if the sizes arent matching
            return np.array(-1)
            
        difference=[] #diffrence between shaplet & signal
        mean_=np.mean(shapletseries)
        shapletlength=len(shapletseries)
        for i in range(0,len(signal)-len(shapletseries)):
            alpha=self.alpha[i]
            diff=np.sum(np.power(signal[i:i+len(shapletseries)]-alpha*shapletseries,2) / ((alpha+alpha**2)*mean_*8*len(shapletseries)))
            difference.append(diff)
        difference=np.array(difference)
        return difference
        
    @staticmethod
    def chi2measure(signal,shaplet:Shaplet):
    #NEED TO CHECK IF THE SHAPLET SERIES IS LOADED IF NOT THEN I LL HAE AN ERROR !!!------------------------------------------------------------------------
        if(len(signal)<len(shaplet)): #By convention the difference is -1 if the sizes arent matching
            return np.array(-1)
            
        difference=[] #diffrence between shaplet & signal
        mean_=np.mean(shaplet.series)
        shapletlength=len(shaplet)
        for i in range(0,len(signal)-len(shaplet)):
            diff=np.sum(np.power(signal[i:i+len(shaplet)]-shaplet.series,2) / (2*mean_*8*len(shaplet)) )
            difference.append(diff)
        difference=np.array(difference)
        return difference

    def __chi2measure(self):
        if self._chi2 is None:
            self._chi2=Comparator.chi2measure(self.data.getcolumn(self.column),self.shaplet)

    def __scaledchi2measuring(self):
        if self._scaledchi2 is None:
            self._scaledchi2=self.scaledchi2measure(self.data.getcolumn(self.column),self.shaplet.series)
    
    def chi2(self):
        self.__chi2measure()
        return self._chi2

    def scaledchi2(self):
        self.__scaledchi2measuring()
        return self._scaledchi2
    
    def __findminimas(self):
        self.__chi2measure()
        
        if np.array_equal(self._chi2,np.array(-1)):
            self._minimas=np.array(-1)
            self._minimaspositions=np.array(-1)
            self._minCut=-1
            self._maxCut=-1
            
        elif ( self._minimaspositions is None ):
            #sig.argrelmin(Comparator.chi2measure(signal,shaplet),order=int(len(shaplet)/2),mode="wrap")[0]
            if(len(self.chi2())<int(len(self.shaplet)/2)):
                self._minimaspositions=np.array([np.argmin(self.chi2())])
                self._minimas=self._chi2[self._minimaspositions]
                self._minCut=min(self._chi2)
                self._maxCut=max(self._chi2)
            else:
                self._minimaspositions=sig.argrelmin(self.chi2(),order=int(len(self.shaplet)/2),mode="wrap")[0]
                self._minimas=self._chi2[self._minimaspositions]
                self._minCut=min(self._minimas)
                self._maxCut=max(self._minimas)
    
    def minimaspositions(self,cut=None):
        self.__findminimas()
        return self._minimaspositions if cut is None else self._minimaspositions[self._minimas<=cut]
    
    def minimas(self,cut=None):
        self.__findminimas()
        return self._minimas if cut is None else self._minimas[self._minimas<=cut]

    def findcut(self,precision=1e-2):
        if not (self._efficientCut is None):
            return

        if(np.array_equal(self._chi2,np.array(-1))):
            self._efficientCut=-1
            return

        if not utilities.noholes(self.__positionsforcut(self._maxCut),len(self.shaplet)):  
            self._efficientCut=-1
            return
        
        self.__findminimas() 
        low=self._minCut
        high=self._maxCut
   #     print(low)
  #      print(high) 
 #       print(precision)
        while high - low > precision:
  #          print(low,end=" it got inside ")
 #           print(high)
            mid = (low + high) / 2
            if utilities.noholes(self.__positionsforcut(mid),len(self.shaplet)):  
                high = mid  # Move left if `foo(mid)` is True
            else:
                low = mid   # Move right if `foo(mid)` is False
                
        self._efficientCut=high

    def __positionsforcut(self,cut):
        return sorted(self._minimaspositions[self._minimas<=cut])

    def min(self):
        self.__findminimas()
        return self._minCut

    def efficient(self):
        self.__findminimas()
        self.findcut()
        return self._efficientCut
    
    def max(self):
        self.__findminimas()
        return self._maxCut

    def coveredarea(self):
        area=0
        minpositions=self.minimaspositions()
        for i in range(len(minpositions)):
            if(i==len(minpositions)-1):
                if(self.signallength-minpositions[i]>=len(self.shaplet)):
                    area+=len(self.shaplet)
                else:
                    area+=self.signallength-minpositions[i]
            else:
                el=minpositions[i]
                nextel=minpositions[i+1]
                if( el+len(self.shaplet) > nextel ):
                    area+=(nextel-el)
                else:
                    area+=len(self.shaplet)
       #     print(area)      
        return area

In [ ]:
class scaledComparator(Comparator):
    def __init__(self,data:Data,column,shaplet:Shaplet):
        super().__init__(data,column,shaplet)
        self.alpha=super()._Comparator__computealpha()
        self._chi2=self.scaledchi2()

In [ ]:
class normalizedComparator(Comparator):
    def __init__(self,data:Data,column,shaplet:Shaplet):
        # definition of a,b,c and d:
        # a*S+b=c*shaplet+d
        super().__init__(data,column,shaplet)
        self.a=self.data.alpha(self.column) 
        self.b=self.data.beta(self.column) 
        self.c=self.shaplet.alpha
        self.d=self.shaplet.beta

        self._chi2= self.normalizedchi2(self.data.getcolumn(self.column),self.shaplet.series)
    
    def normalizedchi2(self,signal,shapletseries):
        #NEED TO CHECK IF THE SHAPLET SERIES IS LOADED IF NOT THEN I LL HAE AN ERROR 
        if(len(signal)<len(shapletseries)): #By convention the difference is -1 if the sizes arent matching
            return np.array(-1)
            
        difference=[] #diffrence between shaplet & signal
        mean_=np.mean(shapletseries)
        shapletlength=len(shapletseries)
        for i in range(0,len(signal)-len(shapletseries)):
            alpha=self.alpha[i]
            #YOU ARE HERE
            #MAKE SURE YOU ARE PROGRAMMING THE RIGHT DIFFERENCE FORMULAE !!
            diff=np.sum(np.power(self.a*signal[i:i+len(shapletseries)]+self.b-self.c*shapletseries-self.d,2) /
                        (( (self.a**2)*np.mean(signal[i:i+len(shapletseries)]) + (self.c **2)*mean_ )*8*len(shapletseries)))
            difference.append(diff)
        difference=np.array(difference)
        return difference

In [ ]:
#ADD TITLE    -----------------------------------------------
#-------------------------------
#---------------------------------
#-------------------------------

class Visualizer:
    def __init__(self):
        pass


    def superpose(self,comparator:Comparator,xlim=None,title=None,cut=None):
        #ADD A TITLE FOR THIS PLOT
        shaplet=comparator.shaplet
        data=comparator.data
        plt.figure()   
        signal=data.getcolumn(comparator.column)
        if(xlim is not None):
            plt.xlim(np.array(xlim)/8)
        if(title is not None):
            plt.title(txt)
        
        minimaspositions=comparator.minimaspositions(cut)
     #   minimaspositions=sig.argrelmin(Comparator.chi2measure(signal,shaplet),order=int(len(shaplet)/2),mode="wrap")[0]
        plt.plot(np.array(range(len(signal)))/8,signal,color='b')  #EVERYTHING IN SECONDS FROM NOW ON !!!
  #                plt.plot(np.array( range(index,index+min(len(shaplet),comparator.signallength)) )/8,
 #                             shaplet.series[:min(len(shaplet),comparator.signallength)],color='r',alpha=0.5)
#
        if (isinstance(comparator,normalizedComparator)):
             for i in range(len(minimaspositions)):
                index=minimaspositions[i]
                if(i==len(minimaspositions)-1 and len(shaplet)+index >=comparator.signallength):
                    plt.plot(np.array(range(index,comparator.signallength))/8,
                                  (comparator.c/comparator.a)*shaplet.series[:comparator.signallength-index]+(comparator.d-comparator.b)/comparator.a,
                             color='r',alpha=0.5)
                else:
                    plt.plot(np.array(range(index,index+len(shaplet)))/8,(comparator.c/comparator.a)*shaplet.series+(comparator.d-comparator.b)/comparator.a,
                             color='r',alpha=0.5)
                    
        else:
            for i in range(len(minimaspositions)):
                index=minimaspositions[i]
                if(i==len(minimaspositions)-1 and len(shaplet)+index >=comparator.signallength):
                    plt.plot(np.array(range(index,comparator.signallength))/8,
                                  comparator.alpha[index]*shaplet.series[:comparator.signallength-index],color='r',alpha=0.5)
                else:
                    plt.plot(np.array(range(index,index+len(shaplet)))/8,comparator.alpha[index]*shaplet.series,color='r',alpha=0.5)
        #plt.xlim(10_000,len(signal))
        plt.xlabel("time in s")
        plt.ylabel("intensity count /s")
        plt.show()
    
    def showchi2(self,comparator:Comparator):
        plt.figure()
        plt.plot(np.array(range(len(comparator.chi2())))/8,comparator.chi2())
        plt.xlabel("window index")
        plt.ylabel("chi^2")
        plt.show()

    def showscaledchi2(self,comparator:Comparator):
        plt.figure()
        plt.plot(np.array(range(len(comparator.scaledchi2())))/8,comparator.scaledchi2())
        plt.xlabel("window index")
        plt.ylabel("scaled chi^2")
        plt.show()

    
    def showlow(self,data:Data):
        plt.figure()
        plt.plot(data.low())
        plt.xlabel("time in s")
        plt.ylabel("intensity count /s")
        plt.show()

    def shownormalizedlow(self,data:Data):
        plt.figure()
        plt.plot(data.alpha("low")*data.low()+data.beta("low"))
        plt.xlabel("time in s")
        plt.ylabel("intensity count /s")
        plt.show()
    
    def show(self,series):
        plt.figure()
        plt.plot(series)
        plt.show()
    

In [ ]:
def minmaxefficient(list_,comparators:list[Comparator]):
    "returns dataframe of min max and efficient cuts for each comparator in comparators"
    cuts=pd.DataFrame(columns=['index','min', 'efficient', 'max'])
  #  print(cuts)
    for index in list_:
 #       print(index)
        cuts.loc[len(cuts)]=[index,comparators[index].min(),comparators[index].efficient(),comparators[index].max()]
    return cuts

In [ ]:
def comparewith(list_,column,shaplet:Shaplet):
    "returns a list of comparators of Data(i) and shaplet where i reperents elements in list_"
    comparators=[]
    for i in list_:
        comparators.append(Comparator(Data(i),column,shaplet))
    return comparators
    
def scaledcomparewith(list_,column,shaplet:Shaplet):
    "returns a list of comparators of Data(i) and shaplet where i reperents elements in list_"
    comparators=[]
    for i in list_:
        comparators.append(scaledComparator(Data(i),column,shaplet))
    return comparators

In [ ]:
# data=Data(290)

# shaplet=Shaplet(181,"low",294,597)

# scaledcomparator=scaledComparator(data,"low",shaplet)

# comparator=Comparator(data,"low",shaplet)

# print(scaledcomparator.minimaspositions()/8)

# print(comparator.minimaspositions()/8)

# plt.figure()
# plt.plot(np.arange(len(scaledcomparator.chi2()))/8,scaledcomparator.chi2())
# plt.show()



# plt.figure()
# plt.plot(comparator.chi2())
# plt.show()



# computed covered area of the shaplet on rho (change the function to count for shaplet that dont end fully) 

# PROGRAM NOW THE DIFFERENT SHAPLETS ESPECIALLY THE SMALL ONES